In [87]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io

In [67]:
songs = pd.read_csv('data/drake-songs.csv')

In [68]:
text = ''

for index, row in songs['lyrics'].iteritems():
    text = text + str(row).lower().replace('|-|', '\n')
    
len(text)

374611

In [81]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 54


In [82]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 124857


In [83]:
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [93]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

Build model...


In [94]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               93696     
_________________________________________________________________
dense_3 (Dense)              (None, 54)                6966      
_________________________________________________________________
activation_3 (Activation)    (None, 54)                0         
Total params: 100,662
Trainable params: 100,662
Non-trainable params: 0
_________________________________________________________________


In [95]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [96]:
def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(
    x, 
    y,
    batch_size=128,
    epochs=30,
    callbacks=[print_callback]
)

Epoch 1/30
124857/124857 [==============================] - 124s 994us/step - loss: 2.0391
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y, nah
never let these niggas ride your "
y, nah
never let these niggas ride your shit it the cand on the say
i gotta shit in my mind on the cand that i got the candes
i gotta spit i me fuck and the cande
i got and the cands it
i got the fucking that i gotta spit it that i got the cand that i gotta scarted that i gotta that i gotta shit it it shit i got the cand it betting for the cand but it that i got the cands cand that i say it that you can't go to the cande that i got and 
----- diversity: 0.5
----- Generating with seed: "y, nah
never let these niggas ride your "
y, nah
never let these niggas ride your back fuck all me cante i should to to you got of the that i have now that i spould the cand
i all me cans
but i wan' put thing girl cands it and go and you say, you can't surting to that but i saying cand go

and i know the money be my ecee friends all back and niggas on my place
damn, you still for the hain for me
i said i'm a mighand and still fuckin' love on my mild
i go niggas party neve
Epoch 8/30
124857/124857 [==============================] - 126s 1ms/step - loss: 1.3567
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "jay and chubbo, shout to gibo
we got san"
jay and chubbo, shout to gibo
we got sant the same and we gone
and i'm a real you get the carse and the six
i need someone that i got a stang
we gon' tire the street things
i don't have to be the start the bitches
i got a lotta like the start to my posile
i got the care that shit the same and the same and the way the carse and the same that i don't want you wanna wanna go
the start is like the best that shit and the same and the start t
----- diversity: 0.5
----- Generating with seed: "jay and chubbo, shout to gibo
we got san"
jay and chubbo, shout to gibo
we got sant go wanna don't really

In [ ]:
def generate_output():
    generated = ''
    #sentence = text[start_index: start_index + Tx]
    #sentence = '0'*Tx
    usr_input = input("Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 0.2)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

In [ ]:
Tx = 40
generate_output()

In [ ]:
import utils.py

In [ ]:
save_model_weights()